<a href="https://colab.research.google.com/github/IvanSandiu/Spark-Data-Analysis/blob/main/DataAnalysis_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analysis using Spark

I started this project by loading employee data from a CSV file into a Spark DataFrame. From there, I explored the data step by step, applying different transformations and running SQL queries to answer questions like average salary, department distributions, and more.

- Task 1: Generate DataFrame from CSV data.
- Task 2: Define a schema for the data.
- Task 3: Display schema of DataFrame.
- Task 4: Create a temporary view.
- Task 5: Execute an SQL query.
- Task 6: Calculate Average Salary by Department.
- Task 7: Filter and Display IT Department Employees.
- Task 8: Add 10% Bonus to Salaries.
- Task 9: Find Maximum Salary by Age.
- Task 10: Self-Join on Employee Data.
- Task 11: Calculate Average Employee Age.
- Task 12: Calculate Total Salary by Department.
- Task 13: Sort Data by Age and Salary.
- Task 14: Count Employees in Each Department.
- Task 15: Filter Employees with the letter o in the Name.

### Prerequisites

Before starting, I made sure my environment had Python and Spark (PySpark) installed, and I also downloaded the CSV file with the employee data.

In [ ]:
!pip install pyspark  findspark wget

import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import wget

findspark.init()

# Creating a SparkContext object
sc = SparkContext.getOrCreate()
# Creating a SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/data/employees.csv")

#### Task 1: Generate a Spark DataFrame from the CSV data

First, I read the employees.csv file into a Spark DataFrame called employees_df so I could start analyzing the data.

In [2]:
employees_df = spark.read.csv("employees.csv", header=True, inferSchema=True)

#### Task 2: Define a schema for the data

Next, I defined a schema for the dataset to make sure Spark interpreted the data types correctly.

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("date_column", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("description", StringType(), True),
    StructField("location", StringType(), True)
])

df1 = spark.read.csv("employees.csv", header=True, schema=schema)

#### Task 3: Display schema of DataFrame

I then displayed the schema of employees_df to confirm that all columns and data types looked correct.

In [ ]:
employees_df.printSchema()

#### Task 4: Create a temporary view

Next, I created a temporary view called employees from the DataFrame so I could run SQL queries directly on the data.


In [5]:
employees_df.createOrReplaceTempView("employees")

#### Task 5: Execute an SQL query

I ran an SQL query on the employees view to retrieve only the employees older than 30. The query returned the filtered records as expected.

In [ ]:
result = spark.sql("SELECT * FROM employees WHERE age > 30")
result.show()

#### Task 6: Calculate Average Salary by Department

Then, I wrote a query to calculate the average salary by department. This gave me a quick look at how salaries are distributed across different teams.

In [ ]:
result = spark.sql("""
    SELECT department, AVG(salary) AS average_salary
    FROM employees
    GROUP BY department
""")
result.show()

#### Task 7: Filter and Display IT Department Employees

To focus specifically on the IT department, I filtered the DataFrame to show only those employees. This helped isolate their information from the rest.

In [ ]:
employeesIT = employees_df.filter("department = 'IT'")
employeesIT.show()

#### Task 8: Add 10% Bonus to Salaries

After that, I added a new column called SalaryAfterBonus. In this column, each salary includes a 10% bonus, letting me see the adjusted earnings for every employee.

In [ ]:
from pyspark.sql.functions import col

employees_df_withBonus = employees_df.withColumn("SalaryAfterBonus", col('salary')*1.1)
employees_df_withBonus.show()

#### Task 9: Find Maximum Salary by Age

I grouped the data by employee age and calculated the maximum salary in each age group. This showed me the top earners for every age.

In [ ]:
from pyspark.sql.functions import max

employees_df_maxSalary = employees_df.orderBy('age').groupBy('age').agg(max("salary").alias("maximum_salary"))
employees_df_maxSalary.show()

#### Task 10: Self-Join on Employee Data

I also performed a self-join on the DataFrame, matching the data with itself using the Emp_No column. This kind of operation can help when comparing employees or checking consistency.

In [ ]:
employees_df_joined = employees_df.join(employees_df, 'Emp_No', 'inner')
employees_df_joined.show()

#### Task 11: Calculate Average Employee Age

To understand the workforce better, I calculated the average age of all employees using an aggregation function.

In [ ]:
from pyspark.sql.functions import avg

employees_df_averageAge = employees_df.agg(avg("age").alias("Average_age"))
employees_df_averageAge.show()

#### Task 12: Calculate Total Salary by Department

I calculated the total salary for each department. This made it easy to see which departments had the largest combined payroll.

In [ ]:
from pyspark.sql.functions import sum

employees_df_departmentSalary = employees_df.groupBy('department').agg(sum("salary").alias("Total_salary"))
employees_df_departmentSalary.show()

#### Task 13: Sort Data by Age and Salary

I sorted the DataFrame by age in ascending order, and within each age group, I sorted by salary in descending order. This way, younger employees appeared first, with the highest earners listed at the top of their age group.

In [ ]:
employees_df_sorted = employees_df.orderBy(col("age").asc(), col("salary").desc())
employees_df_sorted.show()

#### Task 14: Count Employees in Each Department

I counted how many employees belong to each department, which gave me a clear picture of the department sizes.

In [ ]:
from pyspark.sql.functions import count

employees_df_departmentSalary = employees_df.groupBy('department').agg(count("emp_no").alias("Total_employees"))
employees_df_departmentSalary.show()

#### Task 15: Filter Employees with the letter o in the Name

Finally, I applied a filter to select employees whose names contain the letter "o". This was a simple string-based query to check for text conditions in the dataset.

In [ ]:
employeesIT = employees_df.filter(col("emp_name").contains("o"))
employeesIT.show()